# TP06 - Subprocess

Dans le monde du développement Python, il est parfois nécessaire d'interagir avec la ligne de commande du système sur lequel vous travaillez. Cela peut être intéressant lorsque vous souhaitez afficher le contenu d'un dossier, créér un dossier, vérifier les permissions sur certains des fichiers, de pouvoir créér/modifier/démarrer des tâches de la cron table, etc...

Il existe plusieurs manière de faire. Soit via la librairie **os** soit via la librairie **subprocess**. 

Aujourd'hui, nous travaillons en Python3, nous aborderons donc **subprocess**. En parallèle de ce tP, il est fortement recommandé que vous vous informiez sur la librairie **os**.


D'après la documentation officielle, 

"Le module subprocess vous permet de lancer de nouveaux processus, les connecter à des tubes d'entrée/sortie/erreur, et d'obtenir leurs codes de retour."

Vous trouverez la documentation de ce module ici : https://docs.python.org/fr/3.8/library/subprocess.html


La fonction la plus pratique de la librairie subprocess est la fonction **run()**.  Celle-ci repose sur la classe subprocess.  Il existe également la fonction Popen, qui est plus détaillée, mais **run()** suffit dans la plupart des cas.  

```subprocess.run(args, *, stdin=None, input=None, stdout=None, stderr=None, capture_output=False, shell=False, cwd=None, timeout=None, check=False, encoding=None, errors=None, text=None, env=None, universal_newlines=None, **other_popen_kwargs)```

Elle prend beaucoup de paramètres, la plupart facultatifs puisque possédant des valeurs par défaut.  Les paramètres positionnels **args** et suivants représentent le processus à exécuter ainsi que ses paramètres, un token par paramètre.  

<p style="color:red;"><b>Important</b> : 
<ul> <li>Selon le système sur lequel vous travaillez, les commandes shell seront différentes (ex : ls vs dir)</li>
<li> Lorsque vous travaillez avec les subprocess sous Windows, il est nécessaire de définir l'attribut <b>shell</b> comme étant <b>True</b> lorsque vous utilisez les utilitaires 'built-in' de l'invite de commande (ex : dir). </li>
</ul>
    

## 1. Première exécution

**Exercice 1** : Lancez votre première commande avec subprocess pour lister le contenu du dossier courant, en utilisant la méthode subprocess.run()

In [ ]:
import subprocess
#excéuter commande 'ls' pour lister contenu dossier courant. Cette syntaxe car on est sous Windows
subprocess.run("ls", shell=True)

Que constatez-vous ? Que vous retourne cette commande ? Est-ce intéressant ? 

## 2. Afficher le résultat de la commande

Nous avons vu plus haut que la commande **run**, appelée telle qu'elle, renvoie un objet qui n'est pas immédiatement exploitable.  

Avec la commande **run**, deux possibilités s'offrent en fait à vous pour obtenir les sorties de la commande exécutée : 
- soit utiliser **capture_output**, 
- soit définir le **stdout** et le **stderr**. 

Documentez-vous sur ces deux possibilités.  

Notez également l'existance d'un paramètre **universal_newlines**, qui vous sera plus que probablement utile dans vos appels run. A quoi sert-il ? 

**Exercice 2 :**

Reprenez votre exécution de commande de l'exercice 1, et assignez le résultat de votre appel à subprocess à une variable **my_sub**. 
- Affichez l'attribut **stdout** de **my_sub** en console.
- Essayez sans, puis avec le paramètre **capture_output**.  
- Testez également avec et sans **universal_newlines**
- Essayez de rajouter un paramètre à votre commande (ex : ls -la sous Unix).  Quelle syntaxe faut-il utiliser pour cela? ATTENTION : la technique est différente selon que vous utilisez shell=True ou pas.  
- Essayez à présent de lister un fichier qui n'existe pas dans votre répertoire courant (ex : ls tartempion.txt sous Unix, en faisant l'hypothèse que vous n'avez pas de fichier tartempion.txt).  Que contient **stdout** ? Où se trouve le message d'erreur?  

In [ ]:
import subprocess
#exécution sans capture_output
my_sub = subprocess.run("dir", shell=True) #utilisation de dir pour lister les fichiers (remplace la commande ls -la
stdout = my_sub.stdout
print("stdout1:", stdout, "\n" ) #stdout = None, car sortie pas capturée

#Avec capture_output=True
my_sub = subprocess.run("dir", shell=True, capture_output=True)
stdout = my_sub.stdout
print("stdout2:", stdout, "\n") #stdout capturé en bytes (b'')

#avec capture_output=True et universal_newlines=True
my_sub = subprocess.run("dir", shell=True, capture_output=True, universal_newlines=True)
stdout = my_sub.stdout
print("stdout3:", stdout, "\n" ) #stdout capturé en texte lisible
print(my_sub.stderr)

#ajout de paramètres, afficher fichiers cachés 
my_sub = subprocess.run("dir /a", shell=True, capture_output=True, universal_newlines=True)
stdout = my_sub.stdout
print("stdout4:", stdout, "\n" )
print(my_sub.stderr)

#lister un fichier inexistant
my_sub = subprocess.run("dir tartempion.txt",shell=True, capture_output=True, universal_newlines=True)
stdout = my_sub.stdout
print("stdout5:", stdout, "\n" ) #stdout vide si fichier inexistant 
print(my_sub.stderr) #erreur mis dans stderr




**Exercice 3 :** 

Essayez de faire la même chose, non plus en utilisant **capture_output**, mais à l'aide les paramètres stdout et stderr, en leur attribuant comme valeur la variable PIPE de la libraire subprocess. Que représente cette variable? 

In [ ]:
import subprocess
my_sub = subprocess.run("dir", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
print("code de retour: ", my_sub.returncode)
print("Sortie standard: ", my_sub.stdout)
print("Sortie d'erreur: ", my_sub.stderr)

#returncode --> code statu porcess fils, s'il est égale a 0 c'est qu'il est exécuter avec succès.
"""
#regrouper les 2 types d'information, redirection "2>&1 de shell unix"
my_sub = subprocess.run(
    "dir", 
    shell=True, 
    stdout=subprocess.PIPE, 
    stderr=subprocess.STDOUT, # redirige stderr vers stdout
    universal_newlines=True
)
 #Afficher retour combiné
 print("Sortie combinée (stderr + stdout): ", my_sub.stdout)
"""

- Lorsque vous imprimez directement l'objet **my_sub**, que représente l'attribut **returncode** de l'objet affiché? A quoi cela peut-il vous servir? 
- Lorsque vous attribuez la valeur subprocess.PIPE à stdout et stderr du subprocess, ils sont redirigés sur le stdout et stderr du processus courrant (interpréteur python).  Comme nous l'avons vu plus tôt, les messages d'erreurs sont donc séparés de l'output normal.  Comment peut-on faire dans le cas de figure où on souhaite regrouper les deux types d'information (redirection 2>&1 en shell Unix)? 

## 3. Fournir un input à un processus

Nous avons vu comment lire le résultat d'une commande, ou les erreurs lancées par cette dernière.  Nous pouvons également envoyer des données à un processus, en utilisant le paramètre input.  

**Exercice 4 :**

Testez l'envoi d'input sur le stdin d'un processus. Par exemple, en utilisant la commande **grep** sous Linux (**findstr** sous windows), affichez toutes les lignes définissant une interface réseau dans le texte ci-dessous (astuce : elles contiennent toutes le mot flag). 

In [9]:
import subprocess

my_text = """
    lo0: flags=8049<UP,LOOPBACK,RUNNING,MULTICAST> mtu 16384
        options=1203<RXCSUM,TXCSUM,TXSTATUS,SW_TIMESTAMP>
        inet 127.0.0.1 netmask 0xff000000 
        inet6 ::1 prefixlen 128 
        inet6 fe80::1%lo0 prefixlen 64 scopeid 0x1 
        nd6 options=201<PERFORMNUD,DAD>
    gif0: flags=8010<POINTOPOINT,MULTICAST> mtu 1280
    stf0: flags=0<> mtu 1280
    ap1: flags=8802<BROADCAST,SIMPLEX,MULTICAST> mtu 1500
        options=400<CHANNEL_IO>
        ether 3e:22:fb:60:4a:5f 
        media: autoselect
        status: inactive
    en0: flags=8863<UP,BROADCAST,SMART,RUNNING,SIMPLEX,MULTICAST> mtu 1500
        options=400<CHANNEL_IO>
        ether 3c:22:fb:60:4a:5f 
        nd6 options=201<PERFORMNUD,DAD>
        media: autoselect (<unknown type>)
        status: inactive
    en2: flags=8963<UP,BROADCAST,SMART,RUNNING,PROMISC,SIMPLEX,MULTICAST> mtu 1500
        options=460<TSO4,TSO6,CHANNEL_IO>
        ether d2:c0:77:f9:9c:00 
        media: autoselect <full-duplex>
        status: inactive"""

#recherche du mot flags dans le text my_text
process = subprocess.run(["findstr", "flags"], input=my_text.encode(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)

print(process.stdout.decode())



    lo0: flags=8049<UP,LOOPBACK,RUNNING,MULTICAST> mtu 16384
    gif0: flags=8010<POINTOPOINT,MULTICAST> mtu 1280
    stf0: flags=0<> mtu 1280
    ap1: flags=8802<BROADCAST,SIMPLEX,MULTICAST> mtu 1500
    en0: flags=8863<UP,BROADCAST,SMART,RUNNING,SIMPLEX,MULTICAST> mtu 1500
    en2: flags=8963<UP,BROADCAST,SMART,RUNNING,PROMISC,SIMPLEX,MULTICAST> mtu 1500
 




## 4. Pipe entre processus

Puisqu'il est possible de rediriger les flux d'entrées/sortie stdin, stdout et stderr, il est de la même façon possible de connecter différents processus entre eux.  

**Exercice 5 :**

Essayez à présent de chaîner deux appels à la fonction run() : 
- Un premier, appelé **ifconfig**, appelle la commande ifconfig (ipconfig sous Windows). Il redirige sa sortie standard vers un pipe. 
- Un second, appelé **grep**, va utiliser en input le stdout de la commande ifconfig, et la filtrer pour ne garder que la ligne correspondant à votre interface réseau principale (en0, eth0, ...). 




In [15]:
import subprocess

#utiliser ipconfig pour récupérer les informations réseau
process1 = subprocess.run(["ipconfig"], stdout=subprocess.PIPE, stderr=subprocess.PIPE ) #stdout redirigé vers un pipe

#filtrer sortie pour les lignes contenant 'Ethernet' 
process2 = subprocess.run(["findstr", "Ethernet"], input=process1.stdout, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# pour windows, pour afficher les lignes correspondant à l'interface pricipale on utilise 'cp1252' au lieux de 'utf-8'
print(process2.stdout.decode('cp1252'))

Carte Ethernet Ethernet :
Carte Ethernet VMware Network Adapter VMnet1 :
Carte Ethernet VMware Network Adapter VMnet8 :
Carte Ethernet Connexion r‚seau Bluetooth :
Carte Ethernet vEthernet (WSL) :



## 5. Exercices

Nous vous proposons ci-dessous quelques exercices supplémentaires pour vous entraîner avec **subprocess**.  Certains mentionnent textuellement des commandes Unix.  Si vous travaillez sous Windows, remplacez-les par les commandes équivalentes.  Vous pouvez également essayer de travailler avec le WSL (Windows Subsystem for Linux) si vous êtes à l'aise avec ça.  

**Exercice 6 :**

Essayez de lancer un programme quelconque (éditeur de texte, navigateur, ...) depuis votre script Python. Transmettez-lui un paramètre (texte à afficher dans un nouveau document, URL à atteindre, ...)

In [ ]:
import subprocess

#chemin du fichier à ouvrir
file_path = "example.txt"

#lancer Notepad avec le fichier text passer en param
subprocess.run(["notepad", file_path])

url = "https://docs.python.org/3/"

subprocess.run(["start", url], shell=True)

**Exercice 7 :**

Utilisez le module subprocess pour compter le nombre de mot du fichier lorem_ipsum.txt (créez-le vous-même et remplissez le d'un contenu quelconque).  Faites-le de deux manières : 
- En une seule commande avec shell=True (commande cat lorem_ipsum.txt | grep wc)
- Sans shell=True mais avec la combinaison de deux appels de commandes séparés (cat lorem ipsum et wc) 

In [ ]:
import subprocess


#commande pour compter les mots avec shell=True 
command ="type lorem_ipsum.txt | findstr /R /N .*"
# | = Pipe pour rediriger sortie vers commande suivante
# findstr /R /N .* = comme grep, permet de trouver toutes les lignes, /R --> sans /R le findstr traite le fichier comme un chaine littérale, /N --> affiche num ligne correspondants dans la sortie, .* --> correspond à toutes les lignes du fichier
# wc = compte lignes, mots, caractères

#exécuter commande avec shell=True
result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, text=True)
word_count = len(result.stdout.split())

print("Nombre de mots: ", word_count) 


#sans shell=True
#ouvrir le fichier et lire contenu 
with open("lorem_ipsum.txt", "r", encoding="utf-8") as f:
    file_content = f.read()

#utilisation findstr pour récupérer toutes lignes
process = subprocess.run(["findstr", "/R", ".*"], input=file_content, stdout=subprocess.PIPE, text=True)

word_count = len(process.stdout.split())

print("Nombre de mots: ", word_count)

**Exercice 8 :**

Exécutez un traceroute, puis récupérez les IPs des différents sauts dans une liste.  

In [ ]:
import subprocess
import re

#Exécution commande tracert (équivalent de traceroute)
command = ["tracert", "-h", "5", "www.google.com"]

try:
    #Exécuter commande est récupérer la sortie
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    
    #vérification erreur
    if result.returncode != 0:
        print("Erreur lors exécution commande tracert")
    else:
        #Extraire les IPs des sauts via expression régulière
        ip_addresses = re.findall(r"\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b|[a-fA-F0-9:]+", result.stdout)
        
        #afficher les IPs extraites
        print("Liste des IPs des sauts: ", ip_addresses)
except Exception as e:
    print(f"Erreur: {e}")


**Exercice 9 :**

[Bonus] Reprenez l'exercice précédent, et faites en sorte d'afficher les IP au fur et à mesure de l'exécution du traceroute, et pas à la fin.  

Conseil : Cette fois, vous ne pourrez plus utiliser **subprocess.run()**, car il ne permet d'accéder au résultat qu'une fois le processus terminé. Par contre, avec la classe **subprocess.Popen**, l'exécution est non-bloquante, et on peut donc afficher les éléments de stdout au fur et à mesure. 

In [ ]:
import subprocess
import re

#Exécution commande tracert (équivalent de traceroute)
command = ["tracert", "-d","-h", "5", "www.google.com"]
#Extraire les IPs des sauts via expression régulière
ip_pattern = re.compile(r"\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b|(?:[0-9a-fA-F]{1,4}:){2,7}[0-9a-fA-F]{1,4}\b")
try:
    #Exécuter commande est récupérer la sortie
    with subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True) as process:
        print("Traceroute en cours ... \n")
        
        #lire la sortie ligne par ligne
        for line in process.stdout:
            #afficher chaque ligne et rechercher des IPs
            print(line.strip())
            match = ip_pattern.findall(line)
            if match:
                for ip in match:
                    print(f"IP trouvée : {ip}")
                
    #vérification erreur
    return_code = process.wait()
    if return_code != 0:
        print(f"Erreur avec code retour: {return_code}")
        
except Exception as e:
    print(f"Erreur: {e}")